# Part 1

The issue is what happens to the stats of the data generated from an FFT using two measures and several different data configurations. The measures are:

1. The absolute value squared (for Parsavals)
2. The standard deviation of the real and imaginary values separately.

Using standard deviation means that the initial values must be 

Data are assumed to be complex. The different configurations of data are:
1. Filled grid
2. Filled but all imaginary values missing
3. Partially filled grid - random real or imaginary values missing.

Assume the data and fft values have no bias, otherwise what follows is invalid.

I'm interested in what happens when going from FFT->data because the Fourier modes will be controlling the x values in sampling.

I use the FourierOps used by sampling.

In [4]:
import numpy as np
from fourier_ops import FourierOps
dim = 128
fops = FourierOps(dim, dim, 1)        # ntime, nfreq, nant


### Filled grid  

In [17]:
re = np.random.normal(size=(dim, dim))*2.5
im = np.random.normal(size=(dim, dim))*2.5
fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 207131.63340066082 207131.63340066082
sigma real 2.5015840349940244 2.5310531930121685
sigma imag 2.526698875673362 2.496806271368333



From this we can conclude that *both* measures are conserved by the FFT in this case.

### Imag values missing

In [18]:
re = np.random.normal(size=(dim, dim))*2.5
im = np.zeros((dim, dim))
fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 102472.34735803047 102472.34735803047
sigma real 1.7579935867995726 2.5005326889961816
sigma imag 1.7786556444691473 0.0


Parsevals theoem still works, but the rule for the sigmas has to be modified. The conservation law is:

sqrt(sigma_data_real^2 + sigma_data_imag^2)  =  sqrt(sigma_fft_real^2 + sigma_fft_imag^2)

and the real/imag values in the data will have the same sigma.

Or 

sqrt(var_data_real + var_data_imag)  =  sqrt(var_fft_real + var_fft_imag)
    

In [7]:
### Real values missing

In [8]:
re = np.zeros((dim, dim))
im = np.random.normal(size=(dim, dim))
fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 16358.57759241035 16358.57759241035
sigma real 0.7041477953789491 0.0
sigma imag 0.7089115962534093 0.9991330919520007


Similar behaviour to when imag value missing.

### Imag values have different sigma

In [10]:
re = np.zeros((dim, dim))
im = np.random.normal(size=(dim, dim))*2
fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 63939.467715136314 63939.4677151363
sigma real 1.4141083315033676 0.0
sigma imag 1.3794313966004212 1.9753774540750888


### Random real/imag values missing

Different missing locations for real and imag.

In [22]:
re = np.random.normal(size=(dim, dim))+10
im = np.random.normal(size=(dim, dim))

percent_zeroed = 75

x_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))
y_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))
re[x_indices, y_indices] = 0

x_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))
y_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))

im[x_indices, y_indices] = 0


fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 1656258.7128560005 1656258.7128560008
sigma real 9.989975562438186 1.2545138637608815
sigma imag 1.1337886137202196 0.9947599768225804


### Random real/imag values missing

The same locations missing for real and imag.

In [19]:
re = np.random.normal(size=(dim, dim))
im = np.random.normal(size=(dim, dim))

percent_zeroed = 75

x_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))
y_indices = np.random.choice(np.arange(dim), replace=False, size=int(dim * percent_zeroed/100))
re[x_indices, y_indices] = 0

im[x_indices, y_indices] = 0


fft = np.array([re+im*1j])

data = fops.ifft2_normed(fft)

print("Abs squared", np.sum(np.abs(data)**2), np.sum(np.abs(fft)**2))

print("sigma real", np.std(data.real), np.std(fft.real))
print("sigma imag", np.std(data.imag), np.std(fft.imag))

Abs squared 32536.96488488283 32536.96488488283
sigma real 0.9942062343621965 0.9992813058346168
sigma imag 0.9986567859065095 0.9936391024649605


### Conclusion

Parsevals always works but the law for conserving standard deviation from FFT->data is
```
sqrt(var_data_real + var_data_imag)  =  sqrt(var_fft_real + var_fft_imag)
```
and var_data_real will be the same as var_data_imag.

# Part 2

What is the variance of an FFT if the variance 

In [33]:
point_variances = np.abs(np.random.normal(size=(128, 128)))
print(np.mean(point_variances))

def realize(z):
    return np.random.normal(scale=z)

print(np.var(np.vectorize(realize)(x)))




0.8060009506842274
0.9720088586199942
